# Bulk Operations

In this example, we will look at bulk operations using simple saleforce

## Imports

In [67]:
import pandas as pd
from simple_salesforce import Salesforce
import os
import io
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display

load_dotenv()


True

## Authenticate

Always store your credentials in environment variables, and always use a service account.

The `simple-salesforce` library supports multiple authentication methods:

- Username, password, and security token  
- Session ID and instance URL  
- OAuth 2.0 (JWT, web flow, or refresh token)  
- Connected App credentials (via external libraries) 

In [ ]:

sf_domain = os.getenv('sf_domain')
sf_username = os.getenv('sf_username')
sf_password = os.getenv('sf_password')
sf_token = os.getenv('sf_token')


sf = Salesforce(
    username=sf_username,
    password=sf_password,
    security_token=sf_token,
    domain=sf_domain
   
)

## Prepare Data

### Load Data

In [ ]:
df = pd.read_csv('../datasets/parts.csv')
df.sample(5)

###  Clean Data

In [ ]:
#drop duplicate part numbers
df = df.drop_duplicates(subset='Part Number')

In [ ]:
#rename columns to match Salesforce field names
df = df.rename(columns= {
    'Part Number': 'Name',
    'Manufacturer': 'Manufacturer_Name__c',
    'Description': 'inscor__Keyword__c',
    'External Id': 'ExternalId__c'
})
df.sample(1)

## Bulk API Operations

### Common Functions


In [ ]:
def get_bulk2_results(result):
    combined_failed = pd.DataFrame()
    combined_success = pd.DataFrame()
    
    for job in result:
        job_id = job['job_id']
        failed = sf.bulk2.Product2.get_failed_records(job_id)
        success = sf.bulk2.Product2.get_successful_records(job_id)
        
        #since the results are returned as CSV strings, we need to convert them to DataFrames
        success = pd.read_csv(io.StringIO(success))
        failed = pd.read_csv(io.StringIO(failed))
        
        failed['job_id'] = job_id
        success['job_id'] = job_id
        
        combined_failed = pd.concat([combined_failed, failed], ignore_index=True)
        combined_success = pd.concat([combined_success, success], ignore_index=True)
        
        
        return combined_success, combined_failed

### Insert

In [65]:
records=df.to_dict(orient='records')
result = sf.bulk2.Product2.insert(records=records, concurrency=10)
print(result)

[{'numberRecordsFailed': 0, 'numberRecordsProcessed': 7070, 'numberRecordsTotal': 7070, 'job_id': '750ep000003DkzNAAS'}]


In [68]:
import ipywidgets as widgets
from IPython.display import display

success = success.rename(columns={'sf__Id': 'Id'})
csv_path = '../datasets/delete_ids.csv'
success[['Id']].to_csv(csv_path, index=False)

button = widgets.Button(description='Confirm Hard Delete', button_style='danger')

def on_button_click(b):
    print('Running hard delete...')
    result = sf.bulk2.Product2.hard_delete(csv_file=csv_path)
    print('Done:', result)

button.on_click(on_button_click)
display(button)

#Write to CSV in ../dataset — required because simple-salesforce hard_delete asserts csv_file is not None
#Even though records= is accepted, it's not respected internally though a downstream assertion that csv_file is not None for delete operations.
#I'm working on a PR to fix this bug in the simple-salesforce repo
#If you are doing hard delete you will to enable that permission in your profile, do not be silly with that permission

Button(button_style='danger', description='Confirm Hard Delete', style=ButtonStyle())

Running hard delete...
Done: [{'numberRecordsFailed': 0, 'numberRecordsProcessed': 7070, 'numberRecordsTotal': 7070, 'job_id': '750ep000003DlNaAAK'}]


In [ ]:
success = success.rename(
    columns = {'sf__Id': 'Id'}
)
csv_path = '../datasets/delete_ids.csv'
success[['Id']].to_csv(csv_path, index=False)
sf.bulk2.Product2.hard_delete(csv_file=csv_path)

show dynamic get attr
show wait function